In [1]:
from datetime import timedelta
from datetime import datetime
import pandas as pd

## Load data labels

In [2]:
df_dl = pd.read_excel("../Excel Database.xlsx")
df_dl[["Starttime","Endtime"]] = df_dl[["Starttime","Endtime"]].apply(pd.to_datetime)
# df_dl.set_index("SoundID", inplace=True)
df_dl["datetime"] = df_dl["Starttime"] + timedelta(seconds = 6)
df_dl.drop(["UserID","Soundgroup","Soundgroupselection"], axis=1, inplace=True)



print(df_dl.dtypes)
df_dl.head()

SoundID                  object
Starttime        datetime64[ns]
Emotion                  object
Emotionrating             int64
Endtime          datetime64[ns]
datetime         datetime64[ns]
dtype: object


,SoundID,Starttime,Emotion,Emotionrating,Endtime,datetime
0,0319_2,2022-06-15 18:51:47,Fear,5,2022-06-15 18:51:58,2022-06-15 18:51:53
1,0617_2,2022-06-15 18:52:00,Fear,4,2022-06-15 18:52:14,2022-06-15 18:52:06
2,0124_2,2022-06-15 18:52:16,Happiness,4,2022-06-15 18:52:32,2022-06-15 18:52:22
3,0533_2,2022-06-15 18:52:35,Sadness,4,2022-06-15 18:52:49,2022-06-15 18:52:41
4,0194_2,2022-06-15 18:52:50,Fear,5,2022-06-15 18:53:07,2022-06-15 18:52:56


## Load physiological signals

In [3]:
df_hr = pd.read_csv("../hr.csv")
df_hr.drop(["Unnamed: 0","confidence"], axis=1, inplace=True)
df_hr["datetime"] = pd.to_datetime(df_hr["datetime"])

df_skt = pd.read_csv("../skintemp.csv")
df_skt = df_skt.loc[df_skt.index.repeat(12)] #repeat index 12 times 
df_skt.drop("Unnamed: 0", axis=1, inplace=True)
df_skt["datetime"] = pd.to_datetime(df_skt["datetime"])
# df_skt["test_datetime"] = df_skt["datetime"] #used to test the alignment of the physiological signals

df_bo = pd.read_csv("../bo.csv")
df_bo = df_bo.loc[df_bo.index.repeat(12)] #repeat index 12 times 
df_bo.drop("Unnamed: 0", axis=1, inplace=True)
df_bo["datetime"] = pd.to_datetime(df_bo["datetime"])
# df_bo["test_datetime"] = df_bo["datetime"] #used to test the alignment of the physiological signals
df_bo.head()



,datetime,bo_Infrared to Red Signal Ratio
0,2022-06-15 19:09:30,-9
0,2022-06-15 19:09:30,-9
0,2022-06-15 19:09:30,-9
0,2022-06-15 19:09:30,-9
0,2022-06-15 19:09:30,-9


### Merge the signals first

In [4]:
df_phy = pd.merge_asof(df_hr,df_skt, on="datetime",direction="backward", tolerance=pd.Timedelta(seconds=60),allow_exact_matches=True)
df_phy = pd.merge_asof(df_phy,df_bo, on="datetime",direction="nearest", tolerance=pd.Timedelta(seconds=60),allow_exact_matches=True)
df_phy.rename(columns={"bo_Infrared to Red Signal Ratio":"est_02_variation", "temperature":"skin_temp","heartrate":"heart_rate"}, inplace=True)
# df_phy["test_datetime"] = df_phy["datetime"] #used to test the alignment of the physiological signals

print(df_phy.dtypes)

datetime            datetime64[ns]
heart_rate                   int64
skin_temp                  float64
est_02_variation           float64
dtype: object


## Merge Signals and Labels

### Divide data label into 3 (1st dataframe with startime, 2nd with startime + 6 seconds and 3rd with endtime)

In [5]:
#Dataframe with Starttime
df_dl_start = df_dl.drop(["datetime","Endtime"], axis=1)
df_dl_start.rename(columns={"Starttime":"datetime"}, inplace=True)
df_dl_start.head()


,SoundID,datetime,Emotion,Emotionrating
0,0319_2,2022-06-15 18:51:47,Fear,5
1,0617_2,2022-06-15 18:52:00,Fear,4
2,0124_2,2022-06-15 18:52:16,Happiness,4
3,0533_2,2022-06-15 18:52:35,Sadness,4
4,0194_2,2022-06-15 18:52:50,Fear,5


In [6]:
#Dataframe with starttime plus six seconds
df_dl_after6sec = df_dl.drop(["Starttime","Endtime"], axis=1)
df_dl_after6sec.head()

,SoundID,Emotion,Emotionrating,datetime
0,0319_2,Fear,5,2022-06-15 18:51:53
1,0617_2,Fear,4,2022-06-15 18:52:06
2,0124_2,Happiness,4,2022-06-15 18:52:22
3,0533_2,Sadness,4,2022-06-15 18:52:41
4,0194_2,Fear,5,2022-06-15 18:52:56


In [7]:
#Dataframe with endtime
df_dl_end = df_dl.drop(["datetime","Starttime"], axis=1)
df_dl_end.rename(columns={"Endtime":"datetime"},inplace=True)
df_dl_end.head()

,SoundID,Emotion,Emotionrating,datetime
0,0319_2,Fear,5,2022-06-15 18:51:58
1,0617_2,Fear,4,2022-06-15 18:52:14
2,0124_2,Happiness,4,2022-06-15 18:52:32
3,0533_2,Sadness,4,2022-06-15 18:52:49
4,0194_2,Fear,5,2022-06-15 18:53:07


### Merge each Label dataframe with combined physiological signals 

In [25]:
df_research_dataset_1 = pd.merge_asof(df_dl_start, df_phy, on="datetime",direction="nearest", tolerance=pd.Timedelta(seconds=3),allow_exact_matches=True)
# df_research_dataset_1.set_index("SoundID", inplace=True)

df_research_dataset_2 = pd.merge_asof(df_dl_after6sec, df_phy, on="datetime",direction="nearest", tolerance=pd.Timedelta(seconds=3),allow_exact_matches=True)
# df_research_dataset_2.set_index("SoundID", inplace=True)

df_research_dataset_3 = pd.merge_asof(df_dl_end, df_phy, on="datetime",direction="nearest", tolerance=pd.Timedelta(seconds=3),allow_exact_matches=True)
# df_research_dataset_3.set_index("SoundID", inplace=True)


### Create a combined dataset

In [26]:
df_research_dataset = pd.concat([df_research_dataset_1,df_research_dataset_2,df_research_dataset_3])
df_research_dataset.sort_values(by=["SoundID"], inplace=True)
df_research_dataset.head()

,SoundID,datetime,Emotion,Emotionrating,heart_rate,skin_temp,est_02_variation
160,0085_2,2022-06-15 19:49:41,Sadness,1,102.0,1.539936,0.0
160,0085_2,2022-06-15 19:49:53,Sadness,1,NaN,NaN,NaN
160,0085_2,2022-06-15 19:49:47,Sadness,1,103.0,1.539936,0.0
8,0109_2,2022-06-15 18:54:10,Happiness,7,76.0,0.499936,NaN
8,0109_2,2022-06-15 18:54:15,Happiness,7,NaN,NaN,NaN


### Groupby the soundID and datetime

In [29]:
df_research_dataset = df_research_dataset.groupby(["SoundID","datetime"]).mean()
df_research_dataset.sort_values("datetime",inplace=True)
df_research_dataset.head()

Emotionrating  heart_rate  skin_temp  \
SoundID datetime                                                    
0319_2  2022-06-15 18:51:47            5.0        77.0   0.119936   
        2022-06-15 18:51:53            5.0        76.0   0.119936   
        2022-06-15 18:51:58            5.0        76.0   0.259936   
0617_2  2022-06-15 18:52:00            4.0        76.0   0.259936   
        2022-06-15 18:52:06            4.0         NaN        NaN   

                             est_02_variation  
SoundID datetime                               
0319_2  2022-06-15 18:51:47               NaN  
        2022-06-15 18:51:53               NaN  
        2022-06-15 18:51:58               NaN  
0617_2  2022-06-15 18:52:00               NaN  
        2022-06-15 18:52:06               NaN

### Derive statistical Features

In [30]:
print("signal successfully saved")
df_research_dataset.to_csv('../df_research_dataset.csv')



signal successfully saved
